In [ ]:
import warnings; warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2
import logging
import torch
from transformers import *
from pytorch_pretrained_bert import BertAdam
from utils import BertDataset, NERProcessor, InputExampleToTensors, NERTrainer,InputExample
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import pandas as pd
import nltk
import numpy as np

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

model_name = './swe-uncased_L-12_H-768_A-12'
tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=False)
processor = NERProcessor('SUC/', tokenizer, do_lower_case=True)

label_list = processor.get_label_list()
model = BertForTokenClassification.from_pretrained(model_name, num_labels=len(label_list))

In [ ]:
model.load_state_dict(torch.load("./suc_bert_ner.pkl"))

In [ ]:
import numpy as np

x = open("biema_dataset/annons","r")

sent_tokenizer = tokenizer
#print(sent_tokenizer.tokenize(x.read()))
predictions = []

sentence_lower = "Dina egenskaperDu kan arbeta självständigt och är trygg i din yrkesroll"
print(sentence_lower)
example = InputExample("", sentence_lower, label='O')
to_tensors = InputExampleToTensors(tokenizer, max_seq_length=128, label_list=label_list)
input_ids, input_mask, segment_ids, label_id = to_tensors(example)
#input_ids, input_mask, segment_ids, label_id = train_data[2]
model.to('cuda')
tokens_tensor = input_ids.view(1,-1).to('cuda')
segments_tensors = segment_ids.view(1,-1).to('cuda')
model.eval()
#logits = model(tokens_tensor, segments_tensors)
logits = model(tokens_tensor, token_type_ids=segments_tensors)
logits = logits[0].detach().cpu().numpy()

predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
pred_tags = [label_list[p_i] for p in predictions for p_i in p]

print(pred_tags)

np_logits = logits
np.argmax(np_logits, axis=2)
lst = np.argmax(np_logits, axis=2)[0].tolist()
lst = lst[1:]

#print(input_ids)
splitinput = tokenizer.tokenize(sentence_lower)
print(lst)
print(splitinput)
##print(logits)
for num, word in zip(lst, splitinput):
    if num == 4 or num == 5:
        print('{} {}'.format(num, word))

In [ ]:
sentence_lower = 'Smygrustning av raketvapen Av MATS LUNDEGÅRD DN:s korrespondent .'
sentence_lower = sentence_lower.lower()
print(sentence_lower)
example = InputExample("", sentence_lower, label='O')
to_tensors = InputExampleToTensors(tokenizer, max_seq_length=128, label_list=label_list)
input_ids, input_mask, segment_ids, label_id = to_tensors(example)
#input_ids, input_mask, segment_ids, label_id = train_data[2]
model.to('cuda')
tokens_tensor = input_ids.view(1,-1).to('cuda')
segments_tensors = segment_ids.view(1,-1).to('cuda')
model.eval()
logits = model(tokens_tensor, segments_tensors)
res = []
res.extend(logits[0].argmax(-1))
np_logits = logits[0].detach().cpu().numpy()
np.argmax(np_logits, axis=2)
lst = np.argmax(np_logits, axis=2)[0].tolist()
lst = lst[1:]

#print(segment_ids)
splitinput = tokenizer.tokenize(sentence_lower)
#print(lst)
#print(list(zip(lst,splitinput)))
#print(logits)
for num, word in zip(lst, splitinput):
    print(num, word)